In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
import seaborn as sns # for plotting.
import os
from sklearn.preprocessing import StandardScaler, LabelEncoder
import math

In [2]:
filepath = "D:/online dataset csv files preprocessing/merged_u1-50_w001_resampled.csv"
df_original = pd.read_csv(filepath)

In [3]:
df_original

,timestamp,LAx,LAy,LAz,GYx,GYy,GYz,name
0,924313410000000,1.242813,-0.160181,-0.900440,0.194931,-0.004242,0.163818,1
1,924313420000000,1.117246,-0.187921,-1.529893,0.236458,0.076378,0.133293,1
2,924313430000000,0.912846,-0.192566,-1.570929,0.226074,0.012253,0.108246,1
3,924313440000000,0.874858,-0.227208,-1.453511,0.139336,-0.050667,0.085030,1
4,924313450000000,0.834022,-0.282976,-1.292671,-0.013374,-0.109314,0.078308,1
...,...,...,...,...,...,...,...,...
2055043,1368327120000000,-9.502644,-7.178468,4.246279,-2.834417,-0.120428,-0.987857,50
2055044,1368327130000000,-9.443804,-7.698220,3.059675,-2.638940,0.561997,-0.947715,50
2055045,1368327140000000,-8.345459,-6.805815,1.853457,-2.483606,1.225222,-0.954696,50
2055046,1368327150000000,-7.580540,-5.629017,1.559257,-2.429500,1.769765,-1.003565,50


In [4]:

df_test = pd.DataFrame(columns=['timestamp','LAx','LAy','LAz','GYx','GYy','GYz','name'])
df_train = pd.DataFrame(columns=['timestamp','LAx','LAy','LAz','GYx','GYy','GYz','name'])

for i in range(1,51):
    df_original_i = df_original[df_original['name'] == i]
    length = len(df_original_i)
    num_rows_70 = int(length*0.7)
    
    train_row = df_original_i[:num_rows_70]
    test_row = df_original_i[num_rows_70:]
    
    df_train = df_train.append(train_row, ignore_index=True)
    df_test = df_test.append(test_row, ignore_index=True)



C:\Users\szizo\AppData\Local\Temp\ipykernel_11776\4155100195.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(train_row, ignore_index=True)
C:\Users\szizo\AppData\Local\Temp\ipykernel_11776\4155100195.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_test = df_test.append(test_row, ignore_index=True)


In [5]:
 df_train.shape, df_test.shape

((1438511, 8), (616537, 8))

In [6]:
import scipy.stats as stats

Fs = 100
frame_size = Fs*6
hop_size =int( Fs*2)

In [7]:

def get_frames(df, frame_size, hop_size):
    df_framed = pd.DataFrame(columns=['name','mean_LA(x)', 'mean_LA(y)', 'mean_LA(z)', 'mean_GY(x)', 'mean_GY(y)', 'mean_GY(z)','sd_LA(x)', 'sd_LA(y)', 'sd_LA(z)', 'sd_GY(x)', 'sd_GY(y)', 'sd_GY(z)'])
    
    
    for j in range (1,51): 
        filtered_df = df[df['name'] == j]
        filtered_df = filtered_df.drop('name', axis = 1)
        for i in range(0, len(filtered_df) - frame_size, hop_size):
            frame = filtered_df[i: i + frame_size]
            
            new_row = [j
                       ,frame['LAx'].mean(),frame['LAy'].mean(),frame['LAz'].mean()
                       ,frame['GYx'].mean(),frame['GYy'].mean(),frame['GYz'].mean()
                       ,frame['LAx'].std(),frame['LAy'].std(),frame['LAz'].std() 
                       ,frame['GYx'].std(),frame['GYy'].std(),frame['GYz'].std()]


            
            df_framed.loc[len(df_framed)] = new_row
            

    return df_framed

In [8]:
df_framed_train = get_frames(df_train,frame_size,hop_size)
df_framed_test = get_frames(df_test,frame_size,hop_size)
df_framed_test

,name,mean_LA(x),mean_LA(y),mean_LA(z),mean_GY(x),mean_GY(y),mean_GY(z),sd_LA(x),sd_LA(y),sd_LA(z),sd_GY(x),sd_GY(y),sd_GY(z)
0,1.0,1.079440,-0.457812,-0.483374,0.023240,0.067672,-0.243304,6.365418,3.471564,8.022846,1.991897,2.015408,1.022918
1,1.0,1.134933,-0.320877,-0.604904,0.003879,0.107309,-0.209093,6.245376,3.380772,8.099944,1.866264,2.062387,0.970364
2,1.0,0.897598,-0.443790,-0.335117,-0.009742,0.152418,-0.146542,6.307970,3.341179,7.719119,1.816172,1.998866,0.921378
3,1.0,0.880457,-0.484832,-0.394704,0.000221,0.131632,-0.157788,6.302425,3.482027,7.984809,1.813148,1.999263,0.976319
4,1.0,0.795555,-0.704519,-0.424300,0.075614,0.122061,-0.197115,6.410847,3.457457,8.273368,1.977537,2.033018,1.043762
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2944,50.0,-1.050170,-0.868146,1.822672,0.124301,0.051219,-0.074775,3.121951,4.057718,4.445262,1.640452,1.244696,0.772043
2945,50.0,-1.156895,-0.895797,1.794204,0.024099,0.013815,-0.138873,3.391729,4.383545,4.652488,1.708457,1.338528,0.824276
2946,50.0,-1.045716,-0.939412,1.663400,-0.047599,-0.007767,-0.161709,3.494533,4.353627,4.649978,1.730355,1.349232,0.819302
2947,50.0,-0.961321,-0.872502,1.623654,-0.033642,0.004411,-0.152788,3.531289,4.171225,4.540213,1.706072,1.354568,0.823220


In [9]:
df_framed_train.shape, df_framed_test.shape

((7059, 13), (2949, 13))

In [10]:
occurrences = df_framed_train['name'].value_counts()
print(occurrences.mean())

141.18


In [11]:
df_shuffled_train = df_framed_train.sample(frac=1).reset_index(drop=True)
df_shuffled_test = df_framed_test.sample(frac=1).reset_index(drop=True)

In [12]:
df_shuffled_train[df_shuffled_train['name']==11]

,name,mean_LA(x),mean_LA(y),mean_LA(z),mean_GY(x),mean_GY(y),mean_GY(z),sd_LA(x),sd_LA(y),sd_LA(z),sd_GY(x),sd_GY(y),sd_GY(z)
19,11.0,0.263361,0.259125,1.101171,0.020776,0.058218,-0.090960,1.855402,3.136271,1.749251,0.998224,0.921467,0.730145
47,11.0,0.467044,0.242854,0.897524,0.033505,-0.062771,-0.075972,1.959563,2.701920,1.790167,1.018110,0.975306,0.694122
56,11.0,0.301186,0.393924,1.155185,0.079611,0.064499,-0.119602,2.052653,3.338683,1.975258,1.093925,1.049618,0.771498
75,11.0,0.187365,0.437681,1.240059,-0.049865,-0.070456,-0.027284,1.920129,3.074785,1.863215,1.100852,1.003435,0.711016
88,11.0,0.200733,0.232844,1.059527,-0.049409,-0.096776,-0.027848,2.040515,2.986649,1.839282,0.950562,1.030162,0.746255
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6857,11.0,0.304272,0.216266,1.064925,0.005286,0.003275,-0.086510,1.963566,3.119414,1.881393,1.045686,0.972453,0.724259
6871,11.0,0.125531,0.258285,0.989373,-0.063240,-0.081639,-0.043102,1.932657,2.770299,1.738625,0.986922,0.978372,0.668497
6933,11.0,0.156807,0.341372,1.071534,-0.021362,-0.064505,-0.016461,1.845379,2.753179,1.725254,1.021535,0.849984,0.667962
6945,11.0,0.154592,0.290820,1.206679,0.005915,0.027073,-0.081216,2.036168,2.635050,1.634268,1.060998,0.955806,0.747117


In [13]:
X_train = df_shuffled_train.drop('name', axis=1)  # Features (all columns except the target)
y_train = df_shuffled_train['name']  # Labels

In [14]:
X_test = df_shuffled_test.drop('name', axis=1)  # Features (all columns except the target)
y_test = df_shuffled_test['name']  # Labels

In [15]:
# X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size = 0.3, random_state = 0, stratify = y)

In [16]:
X_test

,mean_LA(x),mean_LA(y),mean_LA(z),mean_GY(x),mean_GY(y),mean_GY(z),sd_LA(x),sd_LA(y),sd_LA(z),sd_GY(x),sd_GY(y),sd_GY(z)
0,-0.391545,-0.215855,-0.764483,-0.181137,0.129517,0.034611,2.539754,3.609919,2.583726,1.263964,1.224331,0.576098
1,0.153758,0.238397,0.004166,0.148755,-0.055656,0.022835,2.967976,4.499457,2.721432,1.276351,1.109159,1.086431
2,0.155846,-0.129979,-0.059939,-0.025028,-0.013524,0.028699,2.654698,2.529998,3.101911,1.415217,1.827711,0.660589
3,-0.029061,0.542943,0.476438,-0.075382,-0.147059,-0.058421,3.841190,4.544396,3.585174,1.339897,1.229159,1.248753
4,0.370916,0.582134,-0.236891,0.063679,-0.126240,0.004964,3.200351,4.875717,2.813656,1.336076,1.122552,1.086215
...,...,...,...,...,...,...,...,...,...,...,...,...
2944,-0.011587,0.112663,-0.115003,-0.069434,0.017831,0.064122,2.165875,3.261565,2.237928,1.349515,0.692986,0.491152
2945,0.098317,0.241424,-0.315225,0.101935,-0.021180,0.057282,3.002660,5.277390,3.080345,1.236357,1.306735,1.069747
2946,0.566862,-0.393764,-0.571842,0.091451,-0.003817,0.059589,1.872018,3.332294,2.685304,1.090604,0.768918,0.710311
2947,-0.493279,-0.334817,0.087191,-0.014233,0.049680,-0.066275,2.617776,3.518181,3.652074,1.399949,0.729075,0.547664


In [17]:
occurrences = y_test.value_counts()
print(occurrences.mean())

58.98


In [18]:
X_train.shape, X_test.shape

((7059, 12), (2949, 12))

In [19]:
y_train.shape, y_test.shape

((7059,), (2949,))

In [23]:
#Create a svm Classifier
model = svm.SVC(C = 4, # reg paramater
                kernel='linear', #kernel{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’}, default=’rbf’
               ) # Linear Kernel

#Train the model using the training sets
model.fit(X_train, y_train)


from sklearn import metrics
y_pred = model.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.887419464225161


In [21]:
# df_shuffled_train.to_csv('MEAN_SD_FRAMED_RESAMPLED_merged_u1-50_w001_resampled_differentFrames6_2_NewSplitManual_TRAIN.csv', index = False)
# df_shuffled_test.to_csv('MEAN_SD_FRAMED_RESAMPLED_merged_u1-50_w001_resampled_differentFrames6_2_NewSplitManual_TEST.csv', index = False)